In [1]:
#### DATA TYPES ####

In [1]:
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass

@dataclass
class CookingInstruction:
    index: int
    task: str
    duration: int
    attention: bool
    dependencies: Tuple[int] = ()

@dataclass
class ChefTask:
    instruction_index: int
    when_started: Optional[int]=None
    when_dismissed: Optional[int]=None

@dataclass
class Session:
    recipe: Tuple[CookingInstruction, ...]
    chef_tasks: Dict[str, Tuple[ChefTask, ...]]

############################

In [2]:
state = {}  # global state

def main_page():
    return "enter url!"

def process_url(url: str):
    state["url"] = url
    return "enter names!"

# simulation of the story
main_page()
process_url("actual url")

def parse_cooking_instructions(str) -> List[CookingInstructions]:
    pass

"""
private stuff:
1. alice find recipe
2. alice enters url
3. llm is called, it saves recipe, we upload it.
---------
public stuff:
1. alice goes to main page and sees a large text box
2. alice enters a recipe (i.e. list of CookingInstructions) and presses "enter recipe"
3. alice enters names of chefs for this device "alice", "bob"
4. screen shows two parts: alice sees wash hands, bob sees wash hands, and "done!" buttons
5. they both press "done!" buttons
5. screen shows two parts: alice sees "put eggs to boil", bob sees to not do anything, the algorithm thinks he is useless
2. alice presses "done!"
3. alice sees "chop carrots", and a 5 minute timer for eggs
4. alice presses done! (it took her 3 minutes, as expected, she's on time, she's the best.)
5. put carrots in oven, close door, doesn't have to be on.
6. alice sees just timers
7. alice hears egg timer goes off and sees take out eggs
8. alice dismisses timer and goes to sleep. fuck this recipe.
"""

'\n1. alice goes to main page\n2. alice enters url\n3. alice enters names of chefs for this screen "alice", "bob"\n4. screen shows two parts: alice sees wash hands, bob sees wash hands, and "done!" buttons\n5. they both press "done!" buttons\n5. screen shows two parts: alice sees "put eggs to boil", bob sees to not do anything, the algorithm thinks he is useless\n2. alice presses "done!"\n3. alice sees "chop carrots", and a 5 minute timer for eggs\n4. alice presses done! (it took her 3 minutes, as expected, she\'s on time, she\'s the best.)\n5. put carrots in oven, close door, doesn\'t have to be on.\n6. alice sees just timers\n7. alice hears egg timer goes off and sees take out eggs\n8. alice dismisses timer and goes to sleep. fuck this recipe.\n'

In [3]:
## SAT stuff ##

import tqdm
from subprocess import check_call
#!/usr/bin/env python
# coding: utf-8
# test2
from collections import namedtuple
from pycryptosat import Solver
from itertools import product

cooking_time_unit = 30    # in seconds
assert 60 % cooking_time_unit == 0

Assignment = namedtuple("Assignment", ["attention", "time"])


# Trivial Resorce considerations (for now)
def Resources(v):
    return set([]) #return {"oven"} if "bake" in v else set([])


def recipe2sat(chefs, vertices, edges,a, time_ub, time_unit=cooking_time_unit):
    time_slots = range(int(time_ub)*60 // time_unit + 0)
    print('timeslot[0] ==', time_slots[0])
    
    tuples = []
    for p in chefs:
        for v in vertices:
            #for i in range(len(a[v])):
                for t in time_slots:
                    tuples.append((p, t, v))

    tuple2idx = {tpl: idx for idx, tpl in enumerate(tuples, start=1)}
    #idx2tuple = {idx: tpl for tpl, idx in tuple2idx.items()}

    clauses = []

    print("1/6 process does single high attention task")
    for v, u in tqdm.tqdm(list(product(vertices, vertices))):
        #for i, j in product(range(len(a[v])), range(len(a[u]))):
            #if (u != v or i != j) and a[v][i].Attention and a[u][j].Attention:
            #print(u,v,a[v].attention,a[u].attention)
            if (u != v) and a[v].attention and a[u].attention:
                for t in time_slots:
                    for s in range(t, min(t + a[v].time, time_slots[-1] + 1)):
                        for p in chefs:
                            clauses.append(
                                [-tuple2idx[(p, t, v)], -tuple2idx[(p, s, u)]]
                            )


    print("2/6 resource can do one thing at a time")
    for (v, rv), (u, ru) in tqdm.tqdm(
        list(product([(v, rv) for v in vertices if (rv := Resources(v))], repeat=2))
    ):
        if v != u and ru & rv:
            for p1, p2, t in product(
                chefs, chefs, time_slots
            ):
                for s in range(t, min(t + a[v].time, time_slots[-1] + 1)):
                    clauses.append([-tuple2idx[p1, t, v], -tuple2idx[p2, s, u]])
                                              

    print("3/6 every vertex is done at least once")
    for v in tqdm.tqdm(vertices):
        #for i in range(len(a[v])):
            cls = []
            for p in chefs:
                for t in time_slots:
                    cls.append(tuple2idx[(p, t, v)])
            clauses.append(cls)


    print("4/6 every task is done at most once")
    for p, q in tqdm.tqdm(list(product(chefs, chefs))):
        for t, s in product(time_slots, time_slots):
            if p != q or t != s:
                for v in vertices:
                    #for i in range(len(a[v])):
                        clauses.append([-tuple2idx[(p, t, v)], -tuple2idx[(q, s, v)]])

    #print("5/6 the task are executed serially")
    #for v in tqdm.tqdm(Vertices):
    #    for i in range(len(a[v]) - 1):
    #        for p, q in product(Proc, Proc):
    #            for t, s in product(Times, Times):
    #                if s < t + a[v][i].time:
    #                    clauses.append(
    #                        [-tuple2idx[(p, t, v, i)], -tuple2idx[(q, s, v, i + 1)]]
    #                    )


    print("6/6 the task execution follows the graph structure")
    for v, u in tqdm.tqdm(edges):
        for t, s in product(time_slots, time_slots):
           # if s < t + a[v][len(a[v]) - 1].time:
             if s < t + a[v].time:
                for p, q in product(chefs, chefs):
                    clauses.append(
                        [-tuple2idx[(p, t, v)], -tuple2idx[(q, s, u)]]
                    )
    return clauses, tuples, tuple2idx



def satSolve(clauses, tuple2idx):
    '''Setup a SAT cryptosolver to solve the SAT cooking clauses, If a solution is found, returns the resulting cooking instructions as a list, if a solution is not found, return False'''
    s = Solver()
    for cls in tqdm.tqdm(clauses):
        s.add_clause(cls)
    sat, solution = s.solve()
    
    if sat:
        idx2tuple = {idx: tpl for tpl, idx in tuple2idx.items()}
        chosen_triples = [idx2tuple[i] for i, s in enumerate(solution) if s]
        return chosen_triples
        #print(            "\n".join(f"{p} {t / (60 // unit):.1f} '{v}'.{i}" for p, t, v, i in sorted(IDX)))
    else:
        return False
    
    
def binarysearch(f,lb,ub):
    """the function search for an minimal input x between lb and ub for which f returns the value True"""
    while lb < ub:
        mid = (lb + ub) // 2
        if f(mid):
            ub = mid
        else: 
            lb = mid + 1
    return lb


def run_with_timeout(f, args, timeout, default=None):
    ctx = multiprocessing.get_context('fork')
    q = ctx.Queue()
    def ff(f, args, q):
        ret = f(*args)
        q.put(ret)
        
    p = ctx.Process(target=ff, args=(f, args, q))
    p.start()
    try:
        ret = q.get(timeout=timeout)
    except queue.Empty:
        p.kill()
        return default
    p.join()
    return ret



In [11]:
def next_unstarted_task(session, chef):
    """Return the index in the chef's tasks of the first unstarted instruction if exists, otherwise None 
    *** Maybe better to return the length of the tuple otherwise. Useful for taking the tail of unstarted tasks ***"""
    return ([i for i,task in enumerate(session.chef_tasks[chef]) if task.when_started == None]
            +[len(session.chef_tasks[chef])])[0]
    #return ([i for i,task in enumerate(session.chef_tasks[chef]) if task.when_started == None]+[None])[0]
    

def advance_task_for_chef(session, chef, now):
    i = next_unstarted_task(session, chef)
    if i == len(session.chef_tasks[chef]):
    #if i is None:
        return session
        
    chef_tasks = {chef2 : session.chef_tasks[chef2] for chef2 in session.chef_tasks if chef2 != chef}
    chef_tasks[chef] = (
        session.chef_tasks[chef][:i]
        + ( ChefTask(
                session.chef_tasks[chef][i].instruction_index, now),)
        + session.chef_tasks[chef][i + 1:]
    )
    return Session(session.recipe, chef_tasks)


def remaining_recipe_indices(session):
    return tuple(task.instruction_index for task in sum(
        [session.chef_tasks[chef][next_unstarted_task(session, chef):] for chef in session.chef_tasks],()
    ))

def recompile_remaining_session(session,remaining_indices):
    chefs_list = list(session.chef_tasks.keys())
    #remaining_indices = remaining_recipe_indices(session)
    recipe = session.recipe
    
    # 1. Vertices (indices of all future instructions)
    remaining_instructions = [instr for instr in recipe if instr.index in remaining_indices]
    print("remaining_instructions are", remaining_instructions)
    vertices = [instr.index for instr in remaining_instructions]
    vertex_set = set(vertices)  # for fast lookup

    # 2. Edges: only include dependencies where both source and target are in future_instructions
    edges = []
    time_ub = 0
    for instr in remaining_instructions:
        if instr.attention:
            time_ub += instr.duration
        
        for dep in instr.dependencies:
            if dep in vertex_set:
                edges.append((dep, instr.index))

    # 3. Assignment map: only for vertices in the future
    assignment_map = {
        instr.index: Assignment(instr.attention, instr.duration)
        for instr in remaining_instructions
    }

    
    #4. Setup the SAT equations
    clauses, tuples, tuple2idx = recipe2sat(chefs_list, vertices, edges, assignment_map, time_ub, time_unit=cooking_time_unit)
    
    
    #5. Solve SAT
    chosen_triples = satSolve(clauses, tuple2idx)  
    chosen_triples.sort(key=lambda trip: trip[1])
    # trip = (p,t,v) is a triple where trip[0] = p = chef's_name (aka processor), trip[1] = t = starting time_slot,
    # trip[2] = v = instruction index
    
    return {
        chef: tuple(ChefTask(trip[2])
                    for trip in [trip for trip in chosen_triples if trip[0] == chef])
        for chef in chefs_list
    }


def recompile_session(session,remaining_indices = None):
    if remaining_indices == None:
        remaining_indices = remaining_recipe_indices(session)
        
    future_chef_tasks = recompile_remaining_session(session,remaining_indices)
    print(future_chef_tasks)
    return Session(session.recipe, {
        chef: session.chef_tasks[chef][:next_unstarted_task(session, chef)] + future_chef_tasks[chef] 
        for chef in session.chef_tasks
    })



def new_chef_joined(session, new_chef, now):
    
    if new_chef in session.chef_tasks:
        return session
    
    tasks_plus = session.chef_tasks
    tasks_plus[new_chef] = ()
    
    return advance_task_for_chef(recompile_session(Session(session.recipe, tasks_plus)), new_chef, now)

def chef_leave(session, chef):
    raise NotImplementedError

def dismiss_timer(session, chef, now):
    raise NotImplementedError

def active_timers(session, chef) -> Tuple[ChefTask]:
    raise NotImplementedError

# and more

In [12]:
i1 = CookingInstruction(0, "Chop onions", 60, True, [])
i2 = CookingInstruction(1, "Boil water", 120, False, [0])
i3 = CookingInstruction(2, "Saute onions", 90, True, [0,1])    
i4 = CookingInstruction(3, "4 from scretch", 40, True, []) 
i5 = CookingInstruction(4, "5 from scretch", 50, True, [])
i6 = CookingInstruction(5, "6 from scretch", 60, True, [])

recipe = (i1,i2,i3,i4,i5,i6)

#ChefTasks
A1 = ChefTask(0)
A2 = ChefTask(1)
A3 = ChefTask(3)
A4 = ChefTask(4)
A5 = ChefTask(6)

B1 = ChefTask(2)
B2 = ChefTask(5)

#Session
session = Session(recipe, {'Alice':(A1,A2,A3,A4,A5), 'Bob':(B1,B2)})



In [13]:
session1 = advance_task_for_chef(session, 'Alice', 3333)
session1.chef_tasks['Alice']


(ChefTask(instruction_index=0, when_started=3333, when_dismissed=None),
 ChefTask(instruction_index=1, when_started=None, when_dismissed=None),
 ChefTask(instruction_index=3, when_started=None, when_dismissed=None),
 ChefTask(instruction_index=4, when_started=None, when_dismissed=None),
 ChefTask(instruction_index=6, when_started=None, when_dismissed=None))

In [14]:
session2 = advance_task_for_chef(session1, 'Alice', 7777)
session2.chef_tasks['Alice']

(ChefTask(instruction_index=0, when_started=3333, when_dismissed=None),
 ChefTask(instruction_index=1, when_started=7777, when_dismissed=None),
 ChefTask(instruction_index=3, when_started=None, when_dismissed=None),
 ChefTask(instruction_index=4, when_started=None, when_dismissed=None),
 ChefTask(instruction_index=6, when_started=None, when_dismissed=None))

In [15]:
print(remaining_recipe_indices(session))
print(remaining_recipe_indices(session1))
print(remaining_recipe_indices(session2))

(0, 1, 3, 4, 6, 2, 5)
(2, 5, 1, 3, 4, 6)
(2, 5, 3, 4, 6)


In [16]:
remaining_recipe_indices(session)

(0, 1, 3, 4, 6, 2, 5)

In [17]:
recompile_session(session)

remaining_instructions are [CookingInstruction(index=0, task='Chop onions', duration=60, attention=True, dependencies=[]), CookingInstruction(index=1, task='Boil water', duration=120, attention=False, dependencies=[0]), CookingInstruction(index=2, task='Saute onions', duration=90, attention=True, dependencies=[0, 1]), CookingInstruction(index=3, task='4 from scretch', duration=40, attention=True, dependencies=[]), CookingInstruction(index=4, task='5 from scretch', duration=50, attention=True, dependencies=[]), CookingInstruction(index=5, task='6 from scretch', duration=60, attention=True, dependencies=[])]
timeslot[0] == 0
1/6 process does single high attention task


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 118.48it/s]


2/6 resource can do one thing at a time


0it [00:00, ?it/s]


3/6 every vertex is done at least once


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:04<00:00,  1.03s/it]


6/6 the task execution follows the graph structure


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12686086/12686086 [00:02<00:00, 4804329.68it/s]


{'Alice': (ChefTask(instruction_index=5, when_started=None, when_dismissed=None), ChefTask(instruction_index=4, when_started=None, when_dismissed=None), ChefTask(instruction_index=3, when_started=None, when_dismissed=None), ChefTask(instruction_index=0, when_started=None, when_dismissed=None), ChefTask(instruction_index=1, when_started=None, when_dismissed=None), ChefTask(instruction_index=2, when_started=None, when_dismissed=None)), 'Bob': ()}


Session(recipe=(CookingInstruction(index=0, task='Chop onions', duration=60, attention=True, dependencies=[]), CookingInstruction(index=1, task='Boil water', duration=120, attention=False, dependencies=[0]), CookingInstruction(index=2, task='Saute onions', duration=90, attention=True, dependencies=[0, 1]), CookingInstruction(index=3, task='4 from scretch', duration=40, attention=True, dependencies=[]), CookingInstruction(index=4, task='5 from scretch', duration=50, attention=True, dependencies=[]), CookingInstruction(index=5, task='6 from scretch', duration=60, attention=True, dependencies=[])), chef_tasks={'Alice': (ChefTask(instruction_index=5, when_started=None, when_dismissed=None), ChefTask(instruction_index=4, when_started=None, when_dismissed=None), ChefTask(instruction_index=3, when_started=None, when_dismissed=None), ChefTask(instruction_index=0, when_started=None, when_dismissed=None), ChefTask(instruction_index=1, when_started=None, when_dismissed=None), ChefTask(instruction

In [18]:
session_Joe= new_chef_joined(session,'Joe',1414)

remaining_instructions are [CookingInstruction(index=0, task='Chop onions', duration=60, attention=True, dependencies=[]), CookingInstruction(index=1, task='Boil water', duration=120, attention=False, dependencies=[0]), CookingInstruction(index=2, task='Saute onions', duration=90, attention=True, dependencies=[0, 1]), CookingInstruction(index=3, task='4 from scretch', duration=40, attention=True, dependencies=[]), CookingInstruction(index=4, task='5 from scretch', duration=50, attention=True, dependencies=[]), CookingInstruction(index=5, task='6 from scretch', duration=60, attention=True, dependencies=[])]
timeslot[0] == 0
1/6 process does single high attention task


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<00:00, 80.13it/s]


2/6 resource can do one thing at a time


0it [00:00, ?it/s]


3/6 every vertex is done at least once


...

4/6 every task is done at most once


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:10<00:00,  1.15s/it]


6/6 the task execution follows the graph structure


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27526386/27526386 [00:05<00:00, 4824172.51it/s]


{'Alice': (ChefTask(instruction_index=5, when_started=None, when_dismissed=None), ChefTask(instruction_index=4, when_started=None, when_dismissed=None), ChefTask(instruction_index=3, when_started=None, when_dismissed=None), ChefTask(instruction_index=0, when_started=None, when_dismissed=None), ChefTask(instruction_index=1, when_started=None, when_dismissed=None), ChefTask(instruction_index=2, when_started=None, when_dismissed=None)), 'Bob': (), 'Joe': ()}


In [19]:
session_Joe

Session(recipe=(CookingInstruction(index=0, task='Chop onions', duration=60, attention=True, dependencies=[]), CookingInstruction(index=1, task='Boil water', duration=120, attention=False, dependencies=[0]), CookingInstruction(index=2, task='Saute onions', duration=90, attention=True, dependencies=[0, 1]), CookingInstruction(index=3, task='4 from scretch', duration=40, attention=True, dependencies=[]), CookingInstruction(index=4, task='5 from scretch', duration=50, attention=True, dependencies=[]), CookingInstruction(index=5, task='6 from scretch', duration=60, attention=True, dependencies=[])), chef_tasks={'Alice': (ChefTask(instruction_index=5, when_started=None, when_dismissed=None), ChefTask(instruction_index=4, when_started=None, when_dismissed=None), ChefTask(instruction_index=3, when_started=None, when_dismissed=None), ChefTask(instruction_index=0, when_started=None, when_dismissed=None), ChefTask(instruction_index=1, when_started=None, when_dismissed=None), ChefTask(instruction